In [1]:
#input player, event date
import requests,bs4
from datetime import datetime

#get input of player name and life event date as MM/DD/YYYY
player_tofind='Michael Jordan'
date_lifeevent='06/30/1993'

#get player URL
first_letter=player_tofind[player_tofind.find(' ')+1].lower()
url='https://www.basketball-reference.com/players/'+first_letter+'/'

#wrap in url2soup?
res=requests.get(url)
bbsoup=bs4.BeautifulSoup(res.text,'lxml')
body=bbsoup.select('tbody')

names=body[0].find_all('tr')

for player_index,player in enumerate(names):
    if player_tofind==str(player.find_all('a')[0].contents[0]):
        print player_index
        urlnew='https://www.basketball-reference.com/'+player.find_all('a')[0].attrs['href']
        break

#get season URL
date_month=float(date_lifeevent[0:date_lifeevent.find('/')])
if date_month>9:
    urlyear=float(date_lifeevent[-4:])+1
else:
    urlyear=float(date_lifeevent[-4:])
urlnew=urlnew[0:-5]+'/gamelog/'+str(int(urlyear))+'/'
urlnew

207


'https://www.basketball-reference.com//players/j/jordami01/gamelog/1993/'

In [3]:
res=requests.get(urlnew)
bbsoup=bs4.BeautifulSoup(res.text,'lxml')
body=bbsoup.select('tbody')

games=body[0].find_all('tr')
gsc=[]
for game_index,game in enumerate(games):
    stats=game.find_all('td')
    
    for stat_index,stat in enumerate(stats):
        if (stat.attrs['data-stat']=='game_score') & (len(stat)>0):
            gsc.append([str(stats[1].contents[0].contents[0]),float(stats[stat_index].contents[0])])



In [9]:
# #playoff stats in different table, unparseable in comment
# playoffs=bbsoup.find_all("div",  id="all_pgl_basic_playoffs")
#all below code is just getting original table again
# playoffsmaybe=bbsoup.find_all("div", attrs ={"class":"table_outer_container"})
# playofftable=playoffsmaybe[0].contents[1].contents[1]
# column_headers = [th.getText() for th in playofftable.thead.tr.find_all('th')]
# 
# #get indices
# for index,value in enumerate(column_headers):
#     if value=='GmSc':
#         gsc_index=index
#     elif value=='Date':
#         date_index=index
# 
# 
# games=playofftable.tbody.find_all('tr')
# plgsc=[]
# for game_index,game in enumerate(games):
#     stats=game.find_all('td')
#     
#     for stat_index,stat in enumerate(stats):
#         if (stat.attrs['data-stat']=='game_score') & (len(stat)>0):
#             plgsc.append([str(stats[1].contents[0].contents[0]),float(stats[stat_index].contents[0])])


In [ ]:
# get season BPM for player from advanced stats

# find 10 similar player-seasons

# record distribution of GameScores for those player-seasons
#     in BPM excel file?

# is month before life-event (MBLE) different from player-season distribution

# is MPLE different from month after life event (MALE)?


In [10]:
import pandas as pd
xl = pd.ExcelFile("Box Plus Minus 1_1 Historical for BBRef.xlsx")
df = xl.parse("Historical")
df.head()

AssertionError: 